In [ ]:
%pip install pymongo

In [1]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
from datetime import datetime
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_02_RFC_ESTRUCTURA"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

def validar_rfc_estructura(rfc_base: str, homoclave: str) -> bool:
    """
    Valida la estructura de un RFC de 13 caracteres (base + homoclave)
    según la métrica 1_1_02.
    """
    # Verificamos que los datos de entrada sean válidos
    if not isinstance(rfc_base, str) or not isinstance(homoclave, str):
        return False
        
    # Concatenamos para formar el RFC completo
    rfc_completo = rfc_base.strip() + homoclave.strip()
    
    if len(rfc_completo) != 13:
        return False # No tiene la longitud correcta

    # Esta expresión regular implementa TODOS tus puntos:
    rfc_regex = (
        "^[A-Z][AEIOU][A-Z]{2}"  # Punto 1: 4 letras, 2da es vocal
        "[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|[12][0-9]|3[01])"  # Punto 2: Fecha aammdd
        "[A-Z0-9]{3}$"  # Punto 3: Homoclave alfanumérica de 3 dígitos
    )
    
    if re.match(rfc_regex, rfc_completo):
        return True
    else:
        return False

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_rfc():
    """
    Procesa TODAS las declaraciones de la colección de origen
    y guarda los resultados del RFC en la colección 'metricas'.
    """
    client = None
    
    # Contadores para el resumen final
    total_documentos = 0
    con_rfc_cumple = 0
    con_rfc_no_cumple = 0
    sin_rfc = 0
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"Leyendo de: {SOURCE_COLLECTION_NAME}")
        print(f"Escribiendo en: {METRICS_COLLECTION_NAME}")
        print("--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.02 - RFC) ---")

        # --- B. BUSCAR TODOS LOS DOCUMENTOS ---
        # Proyección optimizada para traer solo el objeto RFC
        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosGenerales.rfc": 1
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]

            # --- C. LEER DATOS Y EJECUTAR MÉTRICA (CON LÓGICA 'SIN_DATO') ---
            rfc_base = None
            homoclave = None
            resultado_metrica = "SIN_DATO"
            
            try:
                # Navegación segura por el objeto anidado
                rfc_object = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {}).get("rfc")
                
                # Verificamos que rfc_object sea un diccionario (como en el JSON)
                if isinstance(rfc_object, dict):
                    rfc_base = rfc_object.get("rfc")
                    homoclave = rfc_object.get("homoClave")
            except AttributeError:
                pass # Se queda como SIN_DATO

            # Validamos que tengamos ambos componentes
            if rfc_base and homoclave and rfc_base.strip() != "" and homoclave.strip() != "":
                if validar_rfc_estructura(rfc_base, homoclave):
                    resultado_metrica = "CUMPLE"
                    con_rfc_cumple += 1
                else:
                    resultado_metrica = "NO_CUMPLE"
                    con_rfc_no_cumple += 1
            else:
                # Si falta rfc_base o homoclave, se considera SIN_DATO
                sin_rfc += 1
                # (resultado_metrica ya es "SIN_DATO" por defecto)

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > RFC Base: '{rfc_base}', Homoclave: '{homoclave}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")
            # --- FIN DE LA SALIDA DE CONSOLA ---

            # --- D. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            
            filtro = { "_id": original_id }
            actualizacion = {
                "$set": {
                    METRIC_ID: resultado_metrica
                }
            }
            
            # Usamos UPSERT=TRUE para crear o actualizar el documento en 'metricas'
            target_collection.update_one(filtro, actualizacion, upsert=True)

            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.02 - RFC):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {con_rfc_cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {con_rfc_no_cumple}")
        print(f"  > Métrica 'SIN_DATO': {sin_rfc}")

    except ConnectionFailure:
        print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e:
        print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_rfc()

Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Leyendo de: all_data_20251021
Escribiendo en: metricas
--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.02 - RFC) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None', Homoclave: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None', Homoclave: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None', Homoclave: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910

KeyboardInterrupt: 